In [ ]:
from google.colab import userdata
access_key = userdata.get('accessId')
secret_key =userdata.get('secretId')

In [1]:
!sudo apt update
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
#Check this site for the latest download link https://www.apache.org/dyn/closer.lua/spark/spark-3.2.1/spark-3.2.1-bin-hadoop3.2.tgz
!wget -q https://dlcdn.apache.org/spark/spark-3.2.1/spark-3.2.1-bin-hadoop3.2.tgz
!tar xf spark-3.2.1-bin-hadoop3.2.tgz
!pip install -q findspark
!pip install pyspark
!pip install py4j


Get:1 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,626 B]
Hit:2 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease [1,581 B]
Get:4 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [119 kB]
Get:5 http://security.ubuntu.com/ubuntu jammy-security InRelease [110 kB]
Hit:6 http://archive.ubuntu.com/ubuntu jammy-backports InRelease
Get:7 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  Packages [872 kB]
Hit:8 https://ppa.launchpadcontent.net/c2d4u.team/c2d4u4.0+/ubuntu jammy InRelease
Hit:9 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:10 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Get:11 http://archive.ubuntu.com/ubuntu jammy-updates/universe amd64 Packages [1,375 kB]
Hit:12 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Get:13 http://archive.ubuntu.com/ubun

In [2]:
import os
import sys
# os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
# os.environ["SPARK_HOME"] = "/content/spark-3.2.1-bin-hadoop3.2"


import findspark
findspark.init()
findspark.find()

import pyspark

from pyspark.sql import DataFrame, SparkSession
from typing import List
import pyspark.sql.types as T
import pyspark.sql.functions as F

spark= SparkSession \
       .builder \
       .appName("Our First Spark Example") \
       .getOrCreate()


spark

In [3]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [53]:
date_of_load = '2024-05-01'

In [4]:
from pyspark.sql.types import StructType, StructField, StringType, IntegerType, DateType, FloatType, TimestampType
from pyspark.sql.functions import *

# **Load Customer Data**

In [12]:
customer_schema = StructType([StructField('customer_id', StringType(), True),\
                              StructField('customer_unique_id', StringType(), True),\
                             StructField('customer_zip_code_prefix', StringType(), True),\
                              StructField('customer_city', StringType(), True),
                              StructField('customer_state', StringType(),True)])

In [33]:
if os.path.isfile(f'/content/gdrive/MyDrive/olist-incremental-load/raw-incremental/{date_of_load}/customers.csv'):
  customer_df = spark.read.schema(customer_schema).option('header', 'true').csv(f'/content/gdrive/MyDrive/olist-incremental-load/raw-incremental/{date_of_load}/customers.csv')
  if os.path.isdir(f'/content/gdrive/MyDrive/olist-incremental-load/processed-incremental/customers'):
    existing_customer_df = spark.read.parquet('/content/gdrive/MyDrive/olist-incremental-load/processed-incremental/customers')
    customer_df = customer_df.join(existing_customer_df, existing_customer_df.customer_id == customer_df.customer_id, 'left_anti')
    print(customer_df.count())
    customer_df.show()
    customer_df.repartition(1).write.option("compression", "snappy")\
    .mode('append').parquet('/content/gdrive/MyDrive/olist-incremental-load/processed-incremental/customers')
  else:
    print(customer_df.count())
    customer_df.show()
    customer_df.repartition(1).write.option("compression", "snappy").mode('overwrite').parquet('/content/gdrive/MyDrive/olist-incremental-load/processed-incremental/customers')
else:
  print('No file found')

59
+--------------------+--------------------+------------------------+-------------------+--------------+
|         customer_id|  customer_unique_id|customer_zip_code_prefix|      customer_city|customer_state|
+--------------------+--------------------+------------------------+-------------------+--------------+
|c8881515a45dd6f97...|6c56bc412d285dc42...|                   22783|     rio de janeiro|            RJ|
|7930549f156eea2b0...|dfe634d4b8e067879...|                   91260|       porto alegre|            RS|
|376b286a6479ddacb...|8bc3746e85755be3c...|                   17202|                jau|            SP|
|6f58e369991106fe6...|def127928e892d48d...|                   53610|           igarassu|            PE|
|a35878bee339b4524...|9726a24d281565192...|                   13052|           campinas|            SP|
|033fab69968b0d690...|fc0bcca3d8d90a812...|                   29278|              arace|            ES|
|397e69077924b1f68...|1d3d94e6941c2196d...|                  

In [34]:
existing_customer_df = spark.read.parquet('/content/gdrive/MyDrive/olist-incremental-load/processed-incremental/customers')
existing_customer_df.count()

99441

In [ ]:
customer_df = spark.read.schema(customer_schema).option('header', 'true').csv('/content/gdrive/MyDrive/raw/customers.csv')

# **Load Order_Items**

In [36]:
order_items_schema = StructType([
    StructField('order_id', StringType(), True),
    StructField('order_item_id', IntegerType(), True),
    StructField('product_id', StringType(), True),
    StructField('seller_id', StringType(), True),
    StructField('shipping_limit_date', TimestampType(), True),
    StructField('price', FloatType(), True),
    StructField('freight_value', FloatType(), True)
])


In [54]:
if os.path.isfile(f'/content/gdrive/MyDrive/olist-incremental-load/raw-incremental/{date_of_load}/order_items.csv'):
  order_items_df = spark.read.schema(order_items_schema).option('header', 'true')\
  .csv(f'/content/gdrive/MyDrive/olist-incremental-load/raw-incremental/{date_of_load}/order_items.csv')

  if os.path.isdir(f'/content/gdrive/MyDrive/olist-incremental-load/processed-incremental/order_items'):
    existing_order_items_df = spark.read.parquet('/content/gdrive/MyDrive/olist-incremental-load/processed-incremental/order_items')
    # Perform left anti join to find rows in new_data that are not in existing_data
    incremental_data = order_items_df.join(existing_order_items_df,
                            on=['order_id', 'order_item_id', 'product_id', 'seller_id', 'shipping_limit_date', 'price',
                                'freight_value'], how='left_anti')\
                                .withColumn('total_price', col('price')*col('order_item_id') + col('freight_value')*col('order_item_id'))
    print(incremental_data.count())
    incremental_data.show()
    incremental_data.repartition(1).write.option("compression", "snappy").mode('append').parquet('/content/gdrive/MyDrive/olist-incremental-load/processed-incremental/order_items')

  else:
    order_items_df = order_items_df.withColumn('total_price', col('price')*col('order_item_id') + col('freight_value')*col('order_item_id'))
    print(order_items_df.count())
    order_items_df.show()
    order_items_df.repartition(1).write.option("compression", "snappy").mode('overwrite').parquet('/content/gdrive/MyDrive/olist-incremental-load/processed-incremental/order_items')
else:
  print('No file found')

62
+--------------------+-------------+--------------------+--------------------+-------------------+------+-------------+-----------+
|            order_id|order_item_id|          product_id|           seller_id|shipping_limit_date| price|freight_value|total_price|
+--------------------+-------------+--------------------+--------------------+-------------------+------+-------------+-----------+
|0096668e5b0b8e965...|            1|55782cb82e0efe052...|c003204e1ab016dfa...|2018-07-13 13:46:36| 166.9|        13.47|     180.37|
|0a92bad75c24fdf33...|            1|2c8e38b2b220b65a3...|c8467937e403e76a8...|2018-08-17 20:25:16|  42.9|        13.64|      56.54|
|0c38c51b8a775bccb...|            1|c3d0861113018ae43...|903037660cf848a71...|2018-08-03 18:05:11| 69.99|        23.21|       93.2|
|103de323ece563a10...|            1|a62e25e09e05e6faf...|634964b17796e6430...|2018-08-24 03:30:00| 105.0|         9.67|     114.67|
|1b3190b2dfa9d789e...|            1|ee406bf28024d9777...|7a67c85e85bb2ce8

In [56]:
existing_order_item_df = spark.read.parquet('/content/gdrive/MyDrive/olist-incremental-load/processed-incremental/order_items')
existing_order_item_df.count()

112650

# **Load Order_payments**

In [57]:
order_payments_schema = StructType([
    StructField('order_id', StringType(), True),
    StructField('payment_sequential', IntegerType(), True),
    StructField('payment_type', StringType(), True),
    StructField('payment_installments', IntegerType(), True),
    StructField('payment_value', FloatType(), True)
])


In [65]:
date_of_load = '2024-05-01'

In [66]:
if os.path.isfile(f'/content/gdrive/MyDrive/olist-incremental-load/raw-incremental/{date_of_load}/order_payments.csv'):
  order_payments_df = spark.read.schema(order_payments_schema).option('header', 'true')\
  .csv(f'/content/gdrive/MyDrive/olist-incremental-load/raw-incremental/{date_of_load}/order_payments.csv')

  if os.path.isdir(f'/content/gdrive/MyDrive/olist-incremental-load/processed-incremental/order_payments'):
    existing_order_payments_df = spark.read.parquet('/content/gdrive/MyDrive/olist-incremental-load/processed-incremental/order_payments')
    # Perform left anti join to find rows in new_data that are not in existing_data
    incremental_data = order_payments_df.join(existing_order_payments_df,
                            on=['order_id', 'payment_sequential','payment_type', 'payment_installments','payment_value'], how='left_anti')

    print(incremental_data.count())
    incremental_data.show()
    incremental_data.repartition(1).write.option("compression", "snappy").mode('append').parquet('/content/gdrive/MyDrive/olist-incremental-load/processed-incremental/order_payments')

  else:
    print(order_payments_df.count())
    order_payments_df.show()
    order_payments_df.repartition(1).write.option("compression", "snappy").mode('overwrite').parquet('/content/gdrive/MyDrive/olist-incremental-load/processed-incremental/order_payments')
else:
  print('No file found')

59
+--------------------+------------------+------------+--------------------+-------------+
|            order_id|payment_sequential|payment_type|payment_installments|payment_value|
+--------------------+------------------+------------+--------------------+-------------+
|3794be706c3088573...|                 1| credit_card|                   3|       139.88|
|34688497162fbb53b...|                 1| credit_card|                   3|       153.55|
|6d0940a8f5fba4756...|                 1| credit_card|                  10|      2455.12|
|920c5e9d5a8a142e7...|                 1|      boleto|                   1|        72.54|
|63d6a91d52c3051f9...|                 1| credit_card|                   1|       177.49|
|587e32dd528769d66...|                 1|      boleto|                   1|       189.18|
|2dd5f6a66c1d82ee0...|                 1|      boleto|                   1|        549.7|
|478116017852df9d1...|                 1| credit_card|                   1|        60.65|
|450cb9

In [67]:
existing_order_payment_df = spark.read.parquet('/content/gdrive/MyDrive/olist-incremental-load/processed-incremental/order_payments')
existing_order_payment_df.count()

103886

# **Load Order Reviews**

In [5]:
order_reviews_schema = StructType([
    StructField('review_id', StringType(), True),
    StructField('order_id', StringType(), True),
    StructField('review_score', IntegerType(), True),
    StructField('review_comment_title', StringType(), True),
    StructField('review_comment_message', StringType(),True),
    StructField('review_creation_date', TimestampType(), True),
    StructField('review_answer_timestamp', TimestampType(), True)
])


In [10]:
date_of_load = '2024-05-01'

In [11]:
if os.path.isfile(f'/content/gdrive/MyDrive/olist-incremental-load/raw-incremental/{date_of_load}/order_reviews.csv'):
  order_reviews_df = spark.read.schema(order_reviews_schema).option('header', 'true')\
  .csv(f'/content/gdrive/MyDrive/olist-incremental-load/raw-incremental/{date_of_load}/order_reviews.csv')

  if os.path.isdir(f'/content/gdrive/MyDrive/olist-incremental-load/processed-incremental/order_reviews'):
    existing_order_reviews_df = spark.read.parquet('/content/gdrive/MyDrive/olist-incremental-load/processed-incremental/order_reviews')
    # Perform left anti join to find rows in new_data that are not in existing_data
    order_reviews = order_reviews_df.join(existing_order_reviews_df,
                            on=['review_id', 'order_id', 'review_score', 'review_comment_title', 'review_creation_date', 'review_answer_timestamp'],
                            how='left_anti')

    print(order_reviews.count())
    order_reviews.show()
    order_reviews.repartition(1).write.option("compression", "snappy").mode('append').parquet('/content/gdrive/MyDrive/olist-incremental-load/processed-incremental/order_reviews')

  else:

    print(order_reviews_df.count())
    order_reviews_df.show()
    order_reviews_df.repartition(1).write.option("compression", "snappy").mode('overwrite').parquet('/content/gdrive/MyDrive/olist-incremental-load/processed-incremental/order_reviews')
else:
  print('No file found')

708
+--------------------+--------------------+------------+--------------------+--------------------+-----------------------+----------------------+
|           review_id|            order_id|review_score|review_comment_title|review_creation_date|review_answer_timestamp|review_comment_message|
+--------------------+--------------------+------------+--------------------+--------------------+-----------------------+----------------------+
|b396ba75350276a6c...|43050a51a257b17d5...|           1|      Lamenetável...|                NULL|                   NULL|             Boa Noite|
|        é lamentável| esta loja que ta...|        NULL| 2018-08-23 00:00:00|                NULL|                   NULL|   2018-08-30 03:02:39|
|5d6f9cddc8335878d...|38fc895ea0a2aa253...|           5|      mega recomendo|                NULL|                   NULL|  Recebi meu produt...|
|   SUPER RECOMENDO."| 2018-08-10 00:00:00|        NULL|                NULL|                NULL|                   NUL

In [12]:
existing_order_review_df = spark.read.parquet('/content/gdrive/MyDrive/olist-incremental-load/processed-incremental/order_reviews')
existing_order_review_df.count()

104804

# **Load Orders**

In [19]:
orders_schema = StructType([
    StructField('order_id', StringType(), True),
    StructField('customer_id',StringType(), True),
    StructField('order_status', StringType(), True),
    StructField('order_purchase_timestamp', TimestampType(), True),
    StructField('order_approved_timestamp', TimestampType(), True),
    StructField('order_delivered_carrier_date', TimestampType(), True),
    StructField('order_delivered_customer_date', TimestampType(), True),
    StructField('order_estimated_delivery_date', TimestampType(), True)
    ])


In [33]:
date_of_load = '2024-05-01'

In [34]:
if os.path.isfile(f'/content/gdrive/MyDrive/olist-incremental-load/raw-incremental/{date_of_load}/orders.csv'):
  orders_df = spark.read.schema(orders_schema).option('header', 'true')\
  .csv(f'/content/gdrive/MyDrive/olist-incremental-load/raw-incremental/{date_of_load}/orders.csv').drop(col('oder_delivered_carrier_date'))

  if os.path.isdir(f'/content/gdrive/MyDrive/olist-incremental-load/processed-incremental/orders'):
    existing_orders_df = spark.read.parquet('/content/gdrive/MyDrive/olist-incremental-load/processed-incremental/orders')
    # Perform left anti join to find rows in new_data that are not in existing_data
    orders = orders_df.join(existing_orders_df,
                            on=['order_id', 'customer_id', 'order_status', 'order_purchase_timestamp', 'order_approved_timestamp','order_delivered_customer_date',
                                'order_estimated_delivery_date'],
                            how='left_anti').filter("order_status = 'delivered' ")

    print(orders.count())
    orders.show()
    orders.repartition(1).write.option("compression", "snappy").mode('append').parquet('/content/gdrive/MyDrive/olist-incremental-load/processed-incremental/orders')

  else:

    print(orders_df.count())
    orders_df.show()
    orders_df.repartition(1).write.option("compression", "snappy").mode('overwrite').parquet('/content/gdrive/MyDrive/olist-incremental-load/processed-incremental/orders')
else:
  print('No file found')

59
+--------------------+--------------------+------------+------------------------+------------------------+-----------------------------+-----------------------------+----------------------------+
|            order_id|         customer_id|order_status|order_purchase_timestamp|order_approved_timestamp|order_delivered_customer_date|order_estimated_delivery_date|order_delivered_carrier_date|
+--------------------+--------------------+------------+------------------------+------------------------+-----------------------------+-----------------------------+----------------------------+
|0c38c51b8a775bccb...|1b8d4b28ab6042710...|   delivered|     2018-07-29 17:56:09|     2018-07-29 18:05:11|          2018-09-01 00:34:24|          2018-08-20 00:00:00|         2018-08-03 12:37:00|
|d5244670e213108e7...|b960433ecd19b8c85...|   delivered|     2018-08-11 20:55:28|     2018-08-11 21:05:14|          2018-09-01 00:36:42|          2018-08-28 00:00:00|         2018-08-13 12:14:00|
|267519bdbe13b2f2

In [35]:
existing_order_df = spark.read.parquet('/content/gdrive/MyDrive/olist-incremental-load/processed-incremental/orders')
existing_order_df.count()

99441

# **Load Products data**

In [43]:
products_schema = StructType([
    StructField('product_id', StringType(), True),
    StructField('product_category', StringType(), True),
    StructField('product_name_lenght', IntegerType(), True),
    StructField('product_description_lenght', IntegerType(), True),
    StructField('product_photos_qty', IntegerType(), True),
    StructField('product_weight_g', IntegerType(), True),
    StructField('product_length_cm', IntegerType(), True),
    StructField('product_height_cm', IntegerType(), True),
    StructField('product_width_cm', IntegerType(), True)
    ])


In [48]:
date_of_load = '2024-04-01'

In [49]:
if os.path.isfile(f'/content/gdrive/MyDrive/olist-incremental-load/raw-incremental/{date_of_load}/products.csv'):

  products_df = spark.read.schema(products_schema).option('header', 'true')\
  .csv(f'/content/gdrive/MyDrive/olist-incremental-load/raw-incremental/{date_of_load}/products.csv')

  products_df = products_df.drop(col('product_photos_qty'), col('product_length_cm'), col('product_height_cm'), col('product_width_cm'))

  products_name_translation_df = spark.read.option('inferSchema', 'true').option('header', 'true')\
      .csv(f'/content/gdrive/MyDrive/olist-incremental-load/raw-incremental/{date_of_load}/product_category_name_translation.csv')

  products_transformed_df = products_df.join(broadcast(products_name_translation_df),\
                                            products_df.product_category == products_name_translation_df.product_category_name)\
                                            .drop(col('product_category_name'))\
                                            .drop(col('product_category')).drop(col('product_name_lenght'))\
                                            .drop(col('product_description_lenght'))
  products_transformed_df.count()

  products_transformed_df.repartition(1).write.mode('overwrite')\
      .parquet('/content/gdrive/MyDrive/olist-incremental-load/processed-incremental/products')
else:
  print('No file found')

No file found


In [47]:
existing_product_df = spark.read.parquet('/content/gdrive/MyDrive/olist-incremental-load/processed-incremental/products')
existing_product_df.count()

32328

# **Load Sellers Data**

In [50]:
sellers_schema = StructType([StructField('seller_id', StringType(), True),
                            StructField('seller_zip_code_prefix', StringType(), True),
                            StructField('seller_city', StringType(), True),
                            StructField('seller_state', StringType(), True)])



In [54]:
date_of_load = '2024-04-01'

In [55]:
if os.path.isfile(f'/content/gdrive/MyDrive/olist-incremental-load/raw-incremental/{date_of_load}/sellers.csv'):

  sellers_df = spark.read.schema(sellers_schema).option('header', 'true')\
  .csv(f'/content/gdrive/MyDrive/olist-incremental-load/raw-incremental/{date_of_load}/sellers.csv')

  sellers_df.count()

  sellers_df.repartition(1).write.mode('overwrite')\
      .parquet('/content/gdrive/MyDrive/olist-incremental-load/processed-incremental/sellers')
else:
  print('No file found')

No file found


In [56]:
existing_seller_df = spark.read.parquet('/content/gdrive/MyDrive/olist-incremental-load/processed-incremental/sellers')
existing_seller_df.count()

3095